# Практическая работа №2. Линейная регрессия с применением библиотеки Scikit-learn

# Import библиотек

In [2]:
import pandas as pd # Библиотека Pandas для работы с табличными данными
import numpy as np # библиотека Numpy для операций линейной алгебры и прочего
import matplotlib.pyplot as plt # библиотека Matplotlib для визуализации
import seaborn as sns # библиотека seaborn для визуализации
import plotly.graph_objects as go # Библиотека Plotly. Модуль "Graph Objects"
import plotly.express as px # Библиотека Plotly. Модуль "Express"

# предварительная обработка числовых признаков
from sklearn.preprocessing import MinMaxScaler# Импортируем нормализацию от scikit-learn
from sklearn.preprocessing import StandardScaler # Импортируем стандартизацию от scikit-learn
from sklearn.preprocessing import PowerTransformer  # Степенное преобразование от scikit-learn
# предварительная обработка категориальных признаков
from sklearn.preprocessing import OneHotEncoder# Импортируем One-Hot Encoding от scikit-learn
from sklearn.preprocessing import OrdinalEncoder# Импортируем Порядковое кодированиеот scikit-learn

from sklearn.pipeline import Pipeline # Pipeline.Не добавить, не убавить

from sklearn.compose import ColumnTransformer # т.н. преобразователь колонок

from sklearn.base import BaseEstimator, TransformerMixin # для создания собственных преобразователей / трансформеров данных

# Загрузка набора данных

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/synrocka/Diabetes-Prediction/main/data/clean_diabetes_prediction_db.csv", delimiter=',')
df

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,False,True,non-smoker,25.19,6.6,140,False
1,Female,54.0,False,False,non-smoker,27.32,6.6,80,False
2,Male,28.0,False,False,non-smoker,27.32,5.7,158,False
3,Female,36.0,False,False,current,23.45,5.0,155,False
4,Male,76.0,True,True,current,20.14,4.8,155,False
...,...,...,...,...,...,...,...,...,...
96123,Female,36.0,False,False,non-smoker,24.60,4.8,145,False
96124,Female,2.0,False,False,non-smoker,17.37,6.5,100,False
96125,Male,66.0,False,False,past_smoker,27.83,5.7,155,False
96126,Female,24.0,False,False,non-smoker,35.42,4.0,100,False


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96128 entries, 0 to 96127
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gender               96128 non-null  object 
 1   age                  96128 non-null  float64
 2   hypertension         96128 non-null  bool   
 3   heart_disease        96128 non-null  bool   
 4   smoking_history      96128 non-null  object 
 5   bmi                  96128 non-null  float64
 6   HbA1c_level          96128 non-null  float64
 7   blood_glucose_level  96128 non-null  int64  
 8   diabetes             96128 non-null  bool   
dtypes: bool(3), float64(3), int64(1), object(2)
memory usage: 4.7+ MB


Списки числовых и категориальных колонок

In [5]:
cat_columns = []
num_columns = []

for column_name in df.columns:
    if ((df[column_name].dtypes == object) or (df[column_name].dtypes == bool)):
        cat_columns += [column_name]
    else:
        num_columns += [column_name]

print('categorical columns:\t ', cat_columns, '\n len = ', len(cat_columns))

print('numerical columns:\t ',  num_columns, '\n len = ', len(num_columns))

categorical columns:	  ['gender', 'hypertension', 'heart_disease', 'smoking_history', 'diabetes'] 
 len =  5
numerical columns:	  ['age', 'bmi', 'HbA1c_level', 'blood_glucose_level'] 
 len =  4


Предварительная обработка из блокнота EDA.ipynb.  
Из обработки исключен показатель 'age', поскольку он является целевым и будет обработан отдельно.

In [6]:
class QuantileReplacer(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.05):
        self.threshold = threshold
        self.quantiles = {}

    def fit(self, X, y=None):
        for col in X.select_dtypes(include='number'):
            low_quantile = X[col].quantile(self.threshold)
            high_quantile = X[col].quantile(1 - self.threshold)
            self.quantiles[col] = (low_quantile, high_quantile)
        return self

    def transform(self, X):
        X_copy = X.copy()
        for col in X.select_dtypes(include='number'):
            low_quantile, high_quantile = self.quantiles[col]
            rare_mask = ((X[col] < low_quantile) | (X[col] > high_quantile))
            if rare_mask.any():
                rare_values = X_copy.loc[rare_mask, col]
                replace_value = np.mean([low_quantile, high_quantile])
                if rare_values.mean() > replace_value:
                    X_copy.loc[rare_mask, col] = high_quantile
                else:
                    X_copy.loc[rare_mask, col] = low_quantile
        return X_copy

In [8]:
cat_pipe_gender_hypertension_heart_diabetes = Pipeline([("encoder", OrdinalEncoder())])
cat_gender = ["gender", "hypertension", "heart_disease", "diabetes"]

cat_pipe_smoking = Pipeline([
    ('encoder', OneHotEncoder(drop='if_binary',
     handle_unknown='ignore', sparse_output=False))
])
cat_smoking = ['smoking_history']

num_pipe_bmi_HbA1c_glucose = Pipeline([
    ('QuantReplace', QuantileReplacer(threshold=0.01, )),
    ('power', PowerTransformer())
])
num_bmi_HbA1c_glucose = ['bmi',
                             'HbA1c_level', 'blood_glucose_level']

# Сделаем отдельно Pipeline с числовыми признаками
preprocessors_num = ColumnTransformer(transformers=[
    ('num_pipe_bmi_HbA1c_glucose',
     num_pipe_bmi_HbA1c_glucose, num_bmi_HbA1c_glucose)
])

# и Pipeline со всеми признаками
preprocessors_all = ColumnTransformer(transformers=[
    ('cat_gender_hypertension_heart_diabetes',
     cat_pipe_gender_hypertension_heart_diabetes, cat_gender),
    ('cat_smoking', cat_pipe_smoking, cat_smoking),
    ('num_pipe_bmi_HbA1c_glucose',
     num_pipe_bmi_HbA1c_glucose, num_bmi_HbA1c_glucose)
])

In [11]:
# Объединяем названия колонок в один список (важен порядок как в ColumnTransformer)
columns_num = np.hstack(num_bmi_HbA1c_glucose)

# Линейная регрессия

In [12]:
from sklearn.linear_model import SGDRegressor # Линейная регрессия с градиентным спуском от scikit-learn

from sklearn.model_selection import train_test_split#  функция разбиения на тренировочную и тестовую выборку
# в исполнении scikit-learn
from sklearn.model_selection import ShuffleSplit # при кросс-валидации случайно перемешиваем данные
from sklearn.model_selection import cross_validate # функция кросс-валидации от Scikit-learn

from sklearn.metrics import mean_squared_error as mse # метрика MSE от Scikit-learn
from sklearn.metrics import r2_score # коэффициент детерминации  от Scikit-learn

from sklearn.metrics import PredictionErrorDisplay # Класс визуализации ошибок модели

Считываем данные, разбиваем на тестовую и тренировочную выборки:

In [13]:
X,y = df.drop(columns = ['age']), df['age']

X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=42)

In [14]:
X_train

,gender,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
48218,Male,False,False,non-smoker,27.32,6.5,100,False
30362,Male,False,False,non-smoker,24.90,6.6,85,False
89600,Female,False,False,past_smoker,40.08,6.2,200,True
38511,Female,False,False,non-smoker,29.26,6.2,160,False
67488,Female,False,False,past_smoker,30.93,6.1,85,False
...,...,...,...,...,...,...,...,...
6265,Female,False,False,non-smoker,20.56,6.0,160,False
54886,Female,False,False,past_smoker,24.34,4.5,85,False
76820,Female,False,False,non-smoker,28.71,6.0,90,False
860,Female,False,False,non-smoker,24.96,6.2,158,False


## Оценим модель с использованием только числовых данных

Преобразуем данные

In [15]:
X_train_prep = preprocessors_num.fit_transform(X_train)

X_val_prep = preprocessors_num.transform(X_val)

Обучаем модель

In [17]:
model = SGDRegressor(random_state = 42)

model.fit(X_train_prep, y_train);

## Подготовим несколько функций для анализа обученной модели

Вытаскивание коэффициентов из модели

In [18]:
def get_coefs(model):
    """Берем веса как атрибуты обученной модели.
    Входные переменные:
    ===========
    model: обученная модель
    """
    B0 = model.intercept_[0]
    B = model.coef_
    return B0, B

Написание модели

In [19]:
def print_model(B0, B, features_names):
    """Написание уравнения модели.
    Входные переменные:
    ===========
    B0: смещение (независимый коэффициент)
    weights: веса признаков
    features_names: список названий признаков
    """
    line = '{:.3f}'.format(B0)
    sign = ['+', '-']
    for p, (fn, b) in enumerate(zip(features_names, B)):
        line = line + sign[int(0.5 * (np.sign(b) - 1))] + '{:.2f}*'.format(np.abs(b)) + fn

    print('Решение/n line')

Визуализация весов в виде столбчатых диаграмм

In [ ]:
def vis_weight(weights, features_names=None, width=1200, height=600):
    """Отрисовка весов.
    Входные переменные:
    ===========
    weights: веса признаков
    features_names: список названий признаков
    """
    numbers = np.arange(0, len(weights))
    if features_names:
        tick_labels = np.hstack(['B0', features_names])
    else:
        tick_labels = ['B' + str(num) for num in numbers]
    fig = go.Figure()
    fig.add_trace(go.Bar(x=numbers[weights<0], y=weights))